In [9]:
from helpers import *

# Get list of r-script DOIs in Harvard dataverse

Get list of dois for all R files in a dataverse (defaulting to Harvard's)
	Parameters
	----------
	dataverse_key : string 
					containing user's dataverse API key
	save : boolean
		   whether or not to save the result as a .txt file
	print_status : boolean
				   whether or not to print status messages
	api_url : string
			  url pointing to the dataverse to get URLs for
	Returns
	-------
	r_dois : list of string
			 dois containing r_files in Harvard dataverse

In [ ]:
all_dois = get_r_dois(dataverse_key, save=True, print_status=True)

In [23]:
# some constants
dataverse_key = "2a301287-c8e8-43f5-9862-cf084b310341"
api_url="https://dataverse.harvard.edu/api/search/"
max_retries=5

# defining some constants
r_file_query = "fileContentType:type/x-r-syntax"

# initialize variables to store current state of scraping
page_num = 0
r_dois = []
failures = 0
numfiles = 0

save=True
print_status=True

In [24]:
#keep requesting until the API returns fewer than 1000 results
while True:
    if print_status:
            print("Requesting page {} from API...".format(page_num))
    try:
            #query the API for 1000 results
            myresults = requests.get(api_url,
                                     params= {"q": r_file_query, "type": "file",
                                              "key": dataverse_key, "start": str(1000 * page_num),
                                              "per_page": str(1000)}).json()['data']['items']
            #print(myresults.json())
            if print_status:
                print("Parsing results from page {}...".format(page_num))
            
            # iterate through results, recording dataset_citations
            for myresult in myresults:
                # extract the DOI (if any) from the result
                doi_match = re.search("(doi:[^,]*)", myresult['dataset_citation'])
                if doi_match:
                    r_dois.append(doi_match.group(1) + '\n')
        #retry if failed to pull data
    except:
        if print_status:
            print("Failed to fetch results for page {}. {} retries left".format(page_num,
                                                                                max_retries - 1 - failures))
        if failures < max_retries:
            failures += 1
            continue
        else:
            break

    # if fewer than 1000 results were returned; we must have reached the end
    if len(myresults) < 1000:
        if print_status:
            print("Reached last page of results. Done.")
            print("Total Number of R Files: {}".format(numfiles + len(myresults)))
        break
    page_num += 1
    numfiles += 1000

Requesting page 0 from API...
Parsing results from page 0...
Requesting page 1 from API...
Parsing results from page 1...
Requesting page 2 from API...
Parsing results from page 2...
Requesting page 3 from API...
Parsing results from page 3...
Requesting page 4 from API...
Parsing results from page 4...
Requesting page 5 from API...
Parsing results from page 5...
Reached last page of results. Done.
Total Number of R Files: 5072


In [30]:
myresults

[{u'checksum': {u'type': u'MD5', u'value': u'unknown'},
  u'dataset_citation': u'Stan Oklobdzija, 2016, "Replication Data for: Closing Down and Cashing In: Extremism and Political Fundraising.", https://doi.org/10.15139/S3/12480, UNC Dataverse',
  u'description': u'',
  u'file_content_type': u'type/x-r-syntax',
  u'file_id': u'2960919',
  u'file_type': u'R Syntax',
  u'md5': u'unknown',
  u'name': u'Data_assembly_code.R',
  u'published_at': u'2016-11-20T22:25:45Z',
  u'size_in_bytes': -1,
  u'type': u'file',
  u'url': u'https://dataverse.harvard.edu/api/access/datafile/2960919'},
 {u'checksum': {u'type': u'MD5', u'value': u'unknown'},
  u'dataset_citation': u'Stan Oklobdzija, 2016, "Replication Data for: Closing Down and Cashing In: Extremism and Political Fundraising.", https://doi.org/10.15139/S3/12480, UNC Dataverse',
  u'description': u'',
  u'file_content_type': u'type/x-r-syntax',
  u'file_id': u'2960932',
  u'file_type': u'R Syntax',
  u'md5': u'unknown',
  u'name': u'Recreation

In [35]:
for myresult in myresults:
    # extract the DOI (if any) from the result
    doi_match = re.search("(doi:[^,]*)", myresult['dataset_citation'])
    print doi_match
    if doi_match:
        r_dois.append(doi_match.group(1) + '\n')

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [34]:
print(r_dois)   

[u'doi:10.1162/isec_a_00321"\n']


In [32]:
 
# remove duplicate DOIs
r_dois = list(set(r_dois))

# if save, then save as .txt file 
if save:
    # remove old output file if one exists
    if os.path.exists('r_dois.txt'):   
        os.remove('r_dois.txt')

    # write dois to file, one-per-line
    with open('r_dois.txt', 'a') as myfile:
        for doi in r_dois:
            myfile.write(doi)
return r_dois

NameError: name 'J' is not defined

In [33]:
all_dois[:5]

[u'doi:10.1162/isec_a_00321"\n']

In [23]:
len(all_dois)

1

In [24]:
# convert to directory names
all_doi_dirs = [doi_to_directory(doi.strip()) for doi in all_dois]

In [25]:
# There are 846 datasets containing a total of 3406 R scripts
len(all_dois)

1

In [26]:
all_doi_dirs

[u'doi--10.1162-isec_a_00321"']